# Google Cloud Platform Setup for Retail ML Application

## Project Information
- **Project ID:** `myretail-460817`
- **Project Number:** `920774451489`
- **Application:** Retail ML with Streamlit Interface

---

## Step 1: Authentication

```bash
gcloud auth login
```

**Description:** Initiates the Google Cloud authentication process to grant gcloud CLI access to your Google Cloud resources.

**What happens:**
- Opens your default web browser
- Redirects to Google's OAuth 2.0 authentication flow
- Prompts you to sign in with your Google account
- Stores authentication credentials locally in `~/.config/gcloud/`
- Enables all subsequent gcloud commands to use your account permissions

**Expected Output:** Browser opens with Google sign-in page, then returns to terminal with confirmation message.

---

## Step 2: Check Current Project Configuration

```bash
gcloud config get-value project
```

**Description:** Displays the currently active Google Cloud project ID from your local gcloud configuration.

**Purpose:** 
- Verify which project is currently set as default
- Useful for troubleshooting configuration issues
- Shows `(unset)` if no project is configured

**Expected Output:** Either a project ID or `(unset)`

---

## Step 3: Set Project Configuration

```bash
gcloud config set project myretail-460817
```

**Description:** Sets `myretail-460817` as the active Google Cloud project for all subsequent gcloud commands.

**What happens:**
- Updates the local gcloud configuration file
- Sets the default project context
- All resources created will belong to this project
- Billing will be associated with this project

**Expected Output:** `Updated property [core/project].`

---

## Step 4: Verify Project Configuration

```bash
gcloud config get-value project
```

**Description:** Confirms that the project has been successfully set by displaying the current active project ID.

**Expected Output:** `myretail-460817`

---

## Step 5: Enable Required Google Cloud APIs

```bash
gcloud services enable cloudbuild.googleapis.com run.googleapis.com artifactregistry.googleapis.com
```

**Description:** Enables three essential APIs required for containerized ML application deployment:

### APIs Being Enabled:

1. **Cloud Build API** (`cloudbuild.googleapis.com`)
   - Builds container images in Google Cloud
   - Provides CI/CD capabilities
   - Supports multi-step build processes

2. **Cloud Run API** (`run.googleapis.com`)
   - Deploys and manages containerized applications
   - Provides serverless container hosting
   - Auto-scales based on traffic

3. **Artifact Registry API** (`artifactregistry.googleapis.com`)
   - Stores and manages container images
   - Provides secure, private Docker registries
   - Integrates with Cloud Build and Cloud Run

**Important Notes:**
- API enablement may take 2-5 minutes
- Billing must be enabled on the project
- Some APIs may have associated costs

**Expected Output:** Confirmation messages for each enabled API.

---

## Step 6: Create Docker Repository in Artifact Registry

```bash
gcloud artifacts repositories create retail-ml-repo \
    --repository-format=docker \
    --location=us-central1 \
    --description="Docker repository for retail ML app"
```

**Description:** Creates a private Docker repository in Google Artifact Registry to store your container images.

### Parameters Explained:

- **Repository Name:** `retail-ml-repo`
- **Format:** `docker` (for container images)
- **Location:** `us-central1` (Iowa, USA - low latency region)
- **Description:** Human-readable identifier for the repository

**What it creates:**
- Secure, private Docker registry
- URL: `us-central1-docker.pkg.dev/myretail-460817/retail-ml-repo`
- Access controlled by IAM permissions
- Vulnerability scanning enabled by default

**Expected Output:** Confirmation that the repository was created successfully.

---

## Step 7: Configure Docker Authentication

```bash
gcloud auth configure-docker us-central1-docker.pkg.dev
```

**Description:** Configures your local Docker installation to authenticate with Google Artifact Registry.

**What happens:**
- Updates Docker configuration file (`~/.docker/config.json`)
- Adds authentication helper for the specified registry
- Enables seamless `docker push` and `docker pull` operations
- Uses your gcloud credentials for Docker operations

**Configuration Added:**
```json
{
  "credHelpers": {
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
```

**Expected Output:** Confirmation that Docker is configured for the registry.

---

## Step 8: Navigate to Project Directory

```bash
cd "/Users/riteshub/Library/CloudStorage/GoogleDrive-riteshbhagwat@gmail.com/My Drive/LocalGDrive/Work/Python WD/Workshops/retail_24"
```

**Description:** Changes the current working directory to your project folder containing the application code and Dockerfiles.

**Directory Contents Should Include:**
- `Dockerfile.base` - Base ML environment image
- `Dockerfile.streamlit` - Streamlit application image
- Application source code
- Requirements files
- Configuration files

---

## Step 9: Build Base Docker Image

```bash
docker build --platform=linux/amd64 -f Dockerfile.base -t retail-ml-base:latest .
```

**Description:** Builds the base Docker image containing your ML environment and dependencies.

### Parameters Explained:

- **`--platform=linux/amd64`:** Ensures x86-64 architecture compatibility with Google Cloud Run
- **`-f Dockerfile.base`:** Specifies which Dockerfile to use for the build
- **`-t retail-ml-base:latest`:** Tags the image with name and version
- **`.`:** Uses current directory as build context

**What happens:**
- Reads instructions from `Dockerfile.base`
- Downloads base images and dependencies
- Installs ML libraries (scikit-learn, pandas, numpy, etc.)
- Creates optimized image layers
- Stores image locally for subsequent builds

**Expected Output:** Build progress with layer information, ending with successful image creation.

---

## Step 10: Build Streamlit Application Image

```bash
docker build --platform=linux/amd64 -f Dockerfile.streamlit -t retail-ml-streamlit:latest .
```

**Description:** Builds the Streamlit application Docker image, typically extending from the base ML image.

### Key Features:
- Uses `Dockerfile.streamlit` for Streamlit-specific configuration
- May inherit from the base ML image created in Step 9
- Includes Streamlit dependencies and application code
- Configures proper port exposure (typically 8080 for Cloud Run)

**Typical Dockerfile.streamlit Contents:**
```dockerfile
FROM retail-ml-base:latest
COPY . /app
WORKDIR /app
EXPOSE 8080
CMD ["streamlit", "run", "app.py", "--server.port=8080", "--server.address=0.0.0.0"]
```

**Expected Output:** Build process showing Streamlit installation and app setup.

---

## Step 11: Verify Image Creation

```bash
docker images | grep streamlit
```

**Description:** Lists all local Docker images and filters for those containing "streamlit" to verify successful image creation.

**Expected Output:**
```
retail-ml-streamlit    latest    abc123def456    2 minutes ago    1.2GB
```

**Information Displayed:**
- Repository name
- Tag
- Image ID
- Creation time
- Image size

---

## Step 12: Tag Image for Artifact Registry

```bash
docker tag retail-ml-streamlit:latest us-central1-docker.pkg.dev/myretail-460817/retail-ml-repo/streamlit:latest
# newretail-460904
```

**Description:** Creates a new tag for your local image that matches the Artifact Registry naming convention.

**Registry URL Breakdown:**
- `us-central1-docker.pkg.dev` - Artifact Registry hostname
- `myretail-460817` - Your project ID
- `retail-ml-repo` - Repository name created in Step 6
- `streamlit:latest` - Image name and tag

**What happens:** Creates an additional tag pointing to the same image, enabling push to Artifact Registry.

---

## Step 13: Push Image to Artifact Registry

```bash
docker push us-central1-docker.pkg.dev/myretail-460817/retail-ml-repo/streamlit:latest
```

**Description:** Uploads your local Streamlit Docker image to Google Artifact Registry.

**What happens:**
- Compresses and uploads image layers
- Stores image securely in Google Cloud
- Makes image available for Cloud Run deployment
- Enables version control and access management

**Upload Process:**
- Large layers are compressed and uploaded in parallel
- Only changed layers are uploaded (layer caching)
- Progress is shown for each layer

**Expected Output:** Upload progress for each layer, ending with successful push confirmation.

---

## Step 14: Deploy to Cloud Run

```bash
gcloud run deploy retail-streamlit-service \
  --image=us-central1-docker.pkg.dev/myretail-460817/retail-ml-repo/streamlit:latest \
  --platform=managed \
  --region=us-central1 \
  --allow-unauthenticated
```

**Description:** Deploys your Streamlit application as a serverless service on Google Cloud Run.

### Parameters Explained:

- **Service Name:** `retail-streamlit-service`
- **Image:** Points to your uploaded Artifact Registry image
- **Platform:** `managed` (fully managed Cloud Run)
- **Region:** `us-central1` (matches your repository location)
- **Authentication:** `--allow-unauthenticated` (public access)

**What happens:**
- Creates a new Cloud Run service
- Pulls the image from Artifact Registry
- Configures auto-scaling (0 to 1000+ instances)
- Provides HTTPS endpoint
- Enables automatic SSL certificate

**Expected Output:**
- Deployment progress
- Service URL (e.g., `https://retail-streamlit-service-xxx-uc.a.run.app`)
- Service configuration details

---

## Final Verification

After deployment, you can verify your setup with:

```bash
# Check Cloud Run services
gcloud run services list

# Check Artifact Registry images
gcloud artifacts docker images list us-central1-docker.pkg.dev/myretail-460817/retail-ml-repo

# Test the deployed application
curl https://your-service-url-here
```

## Summary

You've successfully:
1. ✅ Authenticated with Google Cloud
2. ✅ Configured project: `myretail-460817`
3. ✅ Enabled required APIs
4. ✅ Created Artifact Registry repository
5. ✅ Built and tagged Docker images
6. ✅ Pushed images to the registry
7. ✅ Deployed Streamlit app to Cloud Run

Your retail ML application is now live and accessible via the Cloud Run URL!